In [1]:
import nbimporter
import stats_helper as sh
import preprocessing as pp

import time
import math
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix, cohen_kappa_score

Importing Jupyter notebook from stats_helper.ipynb
Importing Jupyter notebook from preprocessing.ipynb


Kohen Cappa - Kappa or Cohen’s Kappa is like classification accuracy, except that it is normalized at the baseline of random chance on your dataset. It is a more useful measure to use on problems that have an imbalance in the classes (e.g. 70-30 split for classes 0 and 1 and you can achieve 70% accuracy by predicting all instances are for class 0)  
https://machinelearningmastery.com/machine-learning-evaluation-metrics-in-r/  

The kappa statistic, which is a number between -1 and 1. The maximum value means complete agreement; zero or lower means chance agreement.

# Double Exponential Smoothing - Holt's Method

In [2]:
class Holt:
    """
        data - dataset with timestamps
        alpha - float [0.0, 1.0], smoothing parameter
        beta - float [0.0, 1.0], smoothing parameter for trend
    """
    
    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta
        self.result = []
    
    def fit(self, data_train):
        self.data_train = data_train
        
        # First value is same as series
        self.result.append(data_train[0])
        for n in range(1, len(data_train) + 1):
            
            # Initialising level and trend
            if n == 1:
                level, trend = data_train[0], data_train[1] - data_train[0]
            
            # Forecasting the point ahead
            if n >= len(data_train): 
                value = self.result[-1]
            else:
                value = data_train[n]
            
            last_level, level = level, self.alpha*value + (1 - self.alpha)*(level + trend)
            trend = self.beta*(level - last_level) + (1 - self.beta)*trend
            self.result.append(level + trend)

        # Returning the smoothed values (without the forecast)
        return self.result[:-1]
    
    # Returns the forecasted point during the fit
    def predict_one(self):
        return self.result[-1]

    def predict(self, data_test, is_classification):
        predictions = []
        self.result.append(self.data_train[0])
        for n in range(1, len(self.data_train) + len(data_test)):
            if n == 1:
                level, trend = self.data_train[0], self.data_train[1] - self.data_train[0]
            if n >= len(self.data_train): # we are forecasting
                value = data_test[n - len(self.data_train)]
                
                predict_value = self.result[-1]
                
                # Adding it for generating binary returns
                if n == len(self.data_train) and is_classification:
                    predictions.append(predict_value)
                
                predict_last_level, predict_level = level, self.alpha*predict_value + (1 - self.alpha)*(level + trend)
                predict_trend = self.beta*(predict_level - predict_last_level) + (1 - self.beta)*trend
                predictions.append(predict_level + predict_trend)
            else:
                value = self.data_train[n]
                
            last_level, level = level, self.alpha*value + (1 - self.alpha)*(level + trend)
            trend = self.beta*(level - last_level) + (1 - self.beta)*trend
            self.result.append(level + trend)
        
        if is_classification:
            return sh.output_to_binary_indicators(sh.data_daily_returns(predictions))
        
        return predictions  

## Plotting Holt

In [3]:
def plotDoubleExponentialSmoothing(data, alphas, betas):
    """
        Plots double exponential smoothing with different alphas and betas
        
        data - dataset with timestamps
        alphas - list of floats, smoothing parameters for level
        betas - list of floats, smoothing parameters for trend
    """
    
    with plt.style.context('seaborn-white'):    
        plt.figure(figsize=(20, 8))
        for alpha in alphas:
            for beta in betas:
                holt = Holt(alpha, beta)
                plt.plot(holt.fit(data), label="Alpha {}, beta {}".format(alpha, beta))
        plt.plot(data, label = "Actual")
        plt.legend(loc="best")
        plt.axis('tight')
        plt.title("Double Exponential Smoothing - Holt")
        plt.grid(True)
        
# plotDoubleExponentialSmoothing(values, alphas=[0.8, 0.05], betas=[0.8, 0.05])

In [4]:
def plotHolt(data, fit_data):
    plt.figure(figsize=(20, 7))
    plt.plot(range(len(data)), data, 'bo-')
    plt.plot(range(len(fit_data)), fit_data, 'r^--')
    plt.show()

## Finding optimal parameters

In [5]:
def print_metric_values(metric_values):
    print('Alpha \t\t\t Beta \t\t\t Metric')
    for i in range(5): #range(len(metric_values)):
        print(metric_values[i][0], '\t\t\t', metric_values[i][1], '\t\t\t', metric_values[i][2])
    print()

In [13]:
def find_optimal_parameters(data_train, num_splits, alphas, betas, is_classification):
    metric_values = list()
    if is_classification:
        print('Using Accuracy for CV')
    else:
        print('Using Mean Squared Error for CV')
        
    for alpha in alphas:
        for beta in betas:
#             print("Alpha {}, Beta {}".format(alpha, beta))
            holt = Holt(alpha, beta)
            metric_value = sh.statTimeSeriesCV(data_train, num_splits, holt, is_classification)
            metric_values.append([alpha, beta, metric_value])
    
#     print_metric_values(metric_values)
    
    # Sorting the Metric Values
    metric_values.sort(reverse=True, key=lambda x: x[len(metric_values) - 1])
    print_metric_values(metric_values)
    
    return metric_values[0][0], metric_values[0][1]

### RMS

In [7]:
# # Alpha:  0.99 			 Beta:  0.0 			 RMS:  1.778531300846033
# train_data, test_data = series_split(values)
# min_rms = 1000
# min_alpha = 0
# min_beta = 0
# for alpha in np.arange(0.0, 1.0, 0.01):
#     for beta in np.arange(0.0, 1.0, 0.01):
#         holt = Holt(alpha, beta)
#         updated = holt.predict_and_update(series_train=train_data, series_test=test_data)
#         rms = math.sqrt(mean_squared_error(test_data, updated))
#         if rms < min_rms:
#             min_rms = rms
#             min_alpha = alpha
#             min_beta = beta
# print('Alpha: ', min_alpha, '\t\t\t Beta: ', min_beta, '\t\t\t RMS: ', min_rms)

### Returns and Accuracy Score

In [8]:
# ## Alpha:  0.14 		 Beta:  0.86 		 acc:  0.5362872421695951
# train_data, test_data = series_split(values)
# bin_test_data = output_to_binary_indicators(data_daily_returns(train_data[-1:] + test_data))
# max_acc = 0
# max_alpha = 0
# max_beta = 0
# for alpha in np.arange(0.01, 1.0, 0.01):
#     for beta in np.arange(0.0, 1.0, 0.01):
#         holt = Holt(alpha, beta)
#         predictions = holt.predict_and_update(train_data, test_data[:])
#         acc = accuracy_score(bin_test_data, predictions)
#         if acc > max_acc:
#             max_acc = acc
#             max_alpha = alpha
#             max_beta = beta
# print('Alpha: ', max_alpha, '\t\t Beta: ', max_beta, '\t\t acc: ', max_acc)

## Error Metric Values

### Regression Metrics

### Classification Metrics

## Using Holt for Prediction

In [9]:
def forecast(data_train, data_test, alpha, beta, is_classification):
    print('Alpha: ', alpha, '\t Beta: ', beta)
    
    holt = Holt(alpha, beta)
    print('Fitting...')
    holt.fit(data_train)
    
    print('Predicting...') 
    predictions = holt.predict(data_test, is_classification)
    
    if is_classification:
        bin_data_test = sh.output_to_binary_indicators(sh.data_daily_returns(data_train[-1:] + data_test))
        sh.classification_metrics(bin_data_test, predictions)
    else:
        sh.regression_metrics(data_test, predictions)

In [10]:
def get_data_ready(symbol_name, alphas, betas, is_classification):
    start_time = time.time()
    num_splits = 10
    data_train, data_test = sh.prepare_data(symbol_name, train_ratio = 0.8)
    alpha, beta = find_optimal_parameters(data_train, num_splits, alphas, betas, is_classification)
    end_time = time.time()
    
    print('Time taken for Cross Validation:', end_time - start_time)
    return data_train, data_test, alpha, beta

In [11]:
def run_Holt(symbol_name):
    is_classification = False
    alphas = np.arange(0, 1, 0.01)
    betas = np.arange(0, 1, 0.01)
    
    data_train, data_test, alpha, beta = get_data_ready(symbol_name, alphas, betas, is_classification)
    forecast(data_train, data_test, alpha, beta, is_classification = True)

In [12]:
run_Holt(symbol_name = 'AAPL')

Using Mean Squared Error for CV
Alpha 0.0, Beta 0.0
Mean Metric Value:  49.38339575671656 

Alpha 0.0, Beta 0.01
Mean Metric Value:  49.38339575671658 

Alpha 0.0, Beta 0.02
Mean Metric Value:  49.38339575671658 

Alpha 0.0, Beta 0.03
Mean Metric Value:  49.38339575671656 

Alpha 0.0, Beta 0.04
Mean Metric Value:  49.38339575671658 

Alpha 0.0, Beta 0.05
Mean Metric Value:  49.38339575671658 

Alpha 0.0, Beta 0.06
Mean Metric Value:  49.38339575671658 

Alpha 0.0, Beta 0.07
Mean Metric Value:  49.38339575671656 

Alpha 0.0, Beta 0.08
Mean Metric Value:  49.38339575671946 

Alpha 0.0, Beta 0.09
Mean Metric Value:  49.38339575671758 

Alpha 0.0, Beta 0.1
Mean Metric Value:  49.38339575671658 

Alpha 0.0, Beta 0.11
Mean Metric Value:  49.38339575671658 

Alpha 0.0, Beta 0.12
Mean Metric Value:  49.38339575671658 

Alpha 0.0, Beta 0.13
Mean Metric Value:  49.38339575671846 

Alpha 0.0, Beta 0.14
Mean Metric Value:  49.38339575671658 

Alpha 0.0, Beta 0.15
Mean Metric Value:  49.38339575671

Mean Metric Value:  3.0606044204012477 

Alpha 0.01, Beta 0.35000000000000003
Mean Metric Value:  3.0056869520573732 

Alpha 0.01, Beta 0.36
Mean Metric Value:  2.9594288587414246 

Alpha 0.01, Beta 0.37
Mean Metric Value:  2.921622531516099 

Alpha 0.01, Beta 0.38
Mean Metric Value:  2.890712768765529 

Alpha 0.01, Beta 0.39
Mean Metric Value:  2.8645714603672205 

Alpha 0.01, Beta 0.4
Mean Metric Value:  2.8418150032401055 

Alpha 0.01, Beta 0.41000000000000003
Mean Metric Value:  2.8225341100648285 

Alpha 0.01, Beta 0.42
Mean Metric Value:  2.8078955912089505 

Alpha 0.01, Beta 0.43
Mean Metric Value:  2.7989847988980734 

Alpha 0.01, Beta 0.44
Mean Metric Value:  2.7956378412315006 

Alpha 0.01, Beta 0.45
Mean Metric Value:  2.795932328329267 

Alpha 0.01, Beta 0.46
Mean Metric Value:  2.796609350830859 

Alpha 0.01, Beta 0.47000000000000003
Mean Metric Value:  2.7941781055537342 

Alpha 0.01, Beta 0.48
Mean Metric Value:  2.7860961633027923 

Alpha 0.01, Beta 0.49
Mean Metric Val

Mean Metric Value:  2.2337612891789327 

Alpha 0.02, Beta 0.63
Mean Metric Value:  2.218247124890448 

Alpha 0.02, Beta 0.64
Mean Metric Value:  2.1999816581112537 

Alpha 0.02, Beta 0.65
Mean Metric Value:  2.1798337642696257 

Alpha 0.02, Beta 0.66
Mean Metric Value:  2.1586522532152634 

Alpha 0.02, Beta 0.67
Mean Metric Value:  2.137182747968379 

Alpha 0.02, Beta 0.68
Mean Metric Value:  2.1160118736635205 

Alpha 0.02, Beta 0.6900000000000001
Mean Metric Value:  2.0955463385856947 

Alpha 0.02, Beta 0.7000000000000001
Mean Metric Value:  2.076024287645876 

Alpha 0.02, Beta 0.71
Mean Metric Value:  2.0575494712334352 

Alpha 0.02, Beta 0.72
Mean Metric Value:  2.0401357565208906 

Alpha 0.02, Beta 0.73
Mean Metric Value:  2.0237499485796686 

Alpha 0.02, Beta 0.74
Mean Metric Value:  2.008344248119159 

Alpha 0.02, Beta 0.75
Mean Metric Value:  1.993874701180979 

Alpha 0.02, Beta 0.76
Mean Metric Value:  1.980306653269341 

Alpha 0.02, Beta 0.77
Mean Metric Value:  1.96761080070

Mean Metric Value:  1.6543533134363837 

Alpha 0.03, Beta 0.92
Mean Metric Value:  1.6604163172685327 

Alpha 0.03, Beta 0.93
Mean Metric Value:  1.6660458560493852 

Alpha 0.03, Beta 0.9400000000000001
Mean Metric Value:  1.6712740837453461 

Alpha 0.03, Beta 0.9500000000000001
Mean Metric Value:  1.6761289635661147 

Alpha 0.03, Beta 0.96
Mean Metric Value:  1.6806275083060067 

Alpha 0.03, Beta 0.97
Mean Metric Value:  1.684770346464041 

Alpha 0.03, Beta 0.98
Mean Metric Value:  1.688538250790563 

Alpha 0.03, Beta 0.99
Mean Metric Value:  1.6918910632908575 

Alpha 0.04, Beta 0.0
Mean Metric Value:  1.9193574826113702 

Alpha 0.04, Beta 0.01
Mean Metric Value:  1.6573386099305334 

Alpha 0.04, Beta 0.02
Mean Metric Value:  1.6208920001988574 

Alpha 0.04, Beta 0.03
Mean Metric Value:  1.6099683172714225 

Alpha 0.04, Beta 0.04
Mean Metric Value:  1.6108915668720336 

Alpha 0.04, Beta 0.05
Mean Metric Value:  1.6115894208106 

Alpha 0.04, Beta 0.06
Mean Metric Value:  1.60547881178

Mean Metric Value:  1.348515903268499 

Alpha 0.05, Beta 0.22
Mean Metric Value:  1.350576506592963 

Alpha 0.05, Beta 0.23
Mean Metric Value:  1.3513389999900287 

Alpha 0.05, Beta 0.24
Mean Metric Value:  1.3505800362736209 

Alpha 0.05, Beta 0.25
Mean Metric Value:  1.3482529238299956 

Alpha 0.05, Beta 0.26
Mean Metric Value:  1.3444704451530123 

Alpha 0.05, Beta 0.27
Mean Metric Value:  1.3394603722829441 

Alpha 0.05, Beta 0.28
Mean Metric Value:  1.3335114238527228 

Alpha 0.05, Beta 0.29
Mean Metric Value:  1.326925027258288 

Alpha 0.05, Beta 0.3
Mean Metric Value:  1.3199817530311782 

Alpha 0.05, Beta 0.31
Mean Metric Value:  1.3129244087426313 

Alpha 0.05, Beta 0.32
Mean Metric Value:  1.3059549703442075 

Alpha 0.05, Beta 0.33
Mean Metric Value:  1.2992403610172478 

Alpha 0.05, Beta 0.34
Mean Metric Value:  1.2929218493786392 

Alpha 0.05, Beta 0.35000000000000003
Mean Metric Value:  1.2871236736972371 

Alpha 0.05, Beta 0.36
Mean Metric Value:  1.2819580147731984 

Alp

...

Mean Metric Value:  0.7106063842588969 

Alpha 0.9500000000000001, Beta 0.61
Mean Metric Value:  0.7155529888366331 

Alpha 0.9500000000000001, Beta 0.62
Mean Metric Value:  0.7205260828405492 

Alpha 0.9500000000000001, Beta 0.63
Mean Metric Value:  0.7255261418374952 

Alpha 0.9500000000000001, Beta 0.64
Mean Metric Value:  0.7305536511197651 

Alpha 0.9500000000000001, Beta 0.65
Mean Metric Value:  0.7356091065170975 

Alpha 0.9500000000000001, Beta 0.66
Mean Metric Value:  0.7406930151912727 

Alpha 0.9500000000000001, Beta 0.67
Mean Metric Value:  0.7458058964141113 

Alpha 0.9500000000000001, Beta 0.68
Mean Metric Value:  0.7509482823298392 

Alpha 0.9500000000000001, Beta 0.6900000000000001
Mean Metric Value:  0.7561207187029463 

Alpha 0.9500000000000001, Beta 0.7000000000000001
Mean Metric Value:  0.7613237656528131 

Alpha 0.9500000000000001, Beta 0.71
Mean Metric Value:  0.7665579983765016 

Alpha 0.9500000000000001, Beta 0.72
Mean Metric Value:  0.7718240078612209 

Alpha 0

Mean Metric Value:  0.8151352295439873 

Alpha 0.96, Beta 0.8
Mean Metric Value:  0.820813775292713 

Alpha 0.96, Beta 0.81
Mean Metric Value:  0.8265331596900443 

Alpha 0.96, Beta 0.8200000000000001
Mean Metric Value:  0.8322942586921261 

Alpha 0.96, Beta 0.8300000000000001
Mean Metric Value:  0.8380979746518553 

Alpha 0.96, Beta 0.84
Mean Metric Value:  0.8439452371832965 

Alpha 0.96, Beta 0.85
Mean Metric Value:  0.8498370040510164 

Alpha 0.96, Beta 0.86
Mean Metric Value:  0.8557742620873613 

Alpha 0.96, Beta 0.87
Mean Metric Value:  0.8617580281407812 

Alpha 0.96, Beta 0.88
Mean Metric Value:  0.8677893500584166 

Alpha 0.96, Beta 0.89
Mean Metric Value:  0.8738693077062492 

Alpha 0.96, Beta 0.9
Mean Metric Value:  0.8799990140302418 

Alpha 0.96, Beta 0.91
Mean Metric Value:  0.8861796161619817 

Alpha 0.96, Beta 0.92
Mean Metric Value:  0.8924122965724687 

Alpha 0.96, Beta 0.93
Mean Metric Value:  0.8986982742778077 

Alpha 0.96, Beta 0.9400000000000001
Mean Metric Valu

Mean Metric Value:  0.4787401646934451 

Alpha 0.98, Beta 0.09
Mean Metric Value:  0.48309659408573563 

Alpha 0.98, Beta 0.1
Mean Metric Value:  0.4874574995388049 

Alpha 0.98, Beta 0.11
Mean Metric Value:  0.4918231621983099 

Alpha 0.98, Beta 0.12
Mean Metric Value:  0.4961937376020291 

Alpha 0.98, Beta 0.13
Mean Metric Value:  0.5005693686555794 

Alpha 0.98, Beta 0.14
Mean Metric Value:  0.5049502370210476 

Alpha 0.98, Beta 0.15
Mean Metric Value:  0.5093365839446827 

Alpha 0.98, Beta 0.16
Mean Metric Value:  0.5137287156794006 

Alpha 0.98, Beta 0.17
Mean Metric Value:  0.5181270011879434 

Alpha 0.98, Beta 0.18
Mean Metric Value:  0.5225318661749543 

Alpha 0.98, Beta 0.19
Mean Metric Value:  0.526943785634605 

Alpha 0.98, Beta 0.2
Mean Metric Value:  0.5313632760977796 

Alpha 0.98, Beta 0.21
Mean Metric Value:  0.5357908882008702 

Alpha 0.98, Beta 0.22
Mean Metric Value:  0.540227199876849 

Alpha 0.98, Beta 0.23
Mean Metric Value:  0.544672810286039 

Alpha 0.98, Beta 0

Mean Metric Value:  0.6148438471483486 

Alpha 0.99, Beta 0.39
Mean Metric Value:  0.6195949899452428 

Alpha 0.99, Beta 0.4
Mean Metric Value:  0.624366983728887 

Alpha 0.99, Beta 0.41000000000000003
Mean Metric Value:  0.629160410815548 

Alpha 0.99, Beta 0.42
Mean Metric Value:  0.6339758487483604 

Alpha 0.99, Beta 0.43
Mean Metric Value:  0.6388138710428684 

Alpha 0.99, Beta 0.44
Mean Metric Value:  0.643675048004763 

Alpha 0.99, Beta 0.45
Mean Metric Value:  0.6485599476082382 

Alpha 0.99, Beta 0.46
Mean Metric Value:  0.6534691364245077 

Alpha 0.99, Beta 0.47000000000000003
Mean Metric Value:  0.6584031805911006 

Alpha 0.99, Beta 0.48
Mean Metric Value:  0.663362646813582 

Alpha 0.99, Beta 0.49
Mean Metric Value:  0.6683481033923222 

Alpha 0.99, Beta 0.5
Mean Metric Value:  0.6733601212678697 

Alpha 0.99, Beta 0.51
Mean Metric Value:  0.6783992750793276 

Alpha 0.99, Beta 0.52
Mean Metric Value:  0.6834661442309657 

Alpha 0.99, Beta 0.53
Mean Metric Value:  0.688561313

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
